# Sampling

In this notebook, picking up where we left off in the "Getting started" tutorial, we are going to walk through a round of sampling. And we are even going to set up everything in a reproducable manner as good as possible.

## Imports & Settings

First the imports.

In [1]:
import numpy as np
import scipy as sp 
import pandas as pd

import emcee                      # inference and backends for sample storage
from multiprocessing import Pool  # for parallelization of the inference

import lymph

Now some settings, e.g. the name of the HDF5 file we would later like to use.

In [2]:
demo_hdf_file = "./_data/demo.hdf5"

## Set up the Model

First, we will set up the model as we would normally. In contrast to the "Getting started" notebook, we will set up a `Bilateral` model here, but that isn't more complicated. Only the data that needs to be provided to this kind of model needs to have information on the contralateral involvement as well, obviously.

In [3]:
graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV'],
    ('lnl'  , 'I'):       ['II'], 
    ('lnl'  , 'II'):      ['III'],
    ('lnl'  , 'III'):     ['IV'],
    ('lnl'  , 'IV'):      []
}

In [4]:
original_model = lymph.Bilateral(graph)

In [5]:
diagnostic_spsn = {
    "MRI": [0.63, 0.81],
    "PET": [0.86, 0.79]
}
original_model.modalities = diagnostic_spsn

## Generate synthetic data

<div class="alert alert-info">
    Note

    This step can be skipped, as that data is already in the `./_data` directory. But it may also serve as a guide on how to generate synthetic datasets.
</div>

In [6]:
max_t = 10
t = np.arange(max_t + 1)

early_p = 0.3
late_p = 0.7

early_time_dist = sp.stats.binom.pmf(t, max_t, early_p)
late_time_dist = sp.stats.binom.pmf(t, max_t, late_p)
time_dists = {"early": early_time_dist, "late": late_time_dist}

original_model.ipsi.base_probs   = [0.05, 0.2 , 0.12, 0.1 ]
original_model.contra.base_probs = [0.01, 0.06, 0.03, 0.01]
original_model.trans_probs = [0.1, 0.3, 0.2]

In [7]:
synthetic_data = original_model.generate_dataset(
    num_patients=200, 
    stage_dist=[0.6, 0.4], 
    time_dists=time_dists
)
synthetic_data.to_csv("./_data/bilateral.csv", index=False)

Now we load the data into the model instance.

In [8]:
synthetic_data = pd.read_csv("./_data/bilateral.csv", header=[0,1,2])
original_model.patient_data = synthetic_data

## Store the model in an HDF5 file

And before we proceed any further, we store the specifics of this model instance in an HDF5 file. It will basically store the graph, the modalities with their sensitivities & specificities as well as the just loaded data in the HDF5 file and allow us to recreate an instance.

In [9]:
original_model.to_hdf(
    filename=demo_hdf_file, 
    name="original/model"
)

/home/rmnldwg/repos/lymph/.venv/lib/python3.8/site-packages/tables/attributeset.py:291: DataTypeWarning: Unsupported type for attribute 'base_symmetric' in node 'model'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/home/rmnldwg/repos/lymph/.venv/lib/python3.8/site-packages/tables/attributeset.py:291: DataTypeWarning: Unsupported type for attribute 'trans_symmetric' in node 'model'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


## Prepare the likelihood function

In the utilities of the `lymph` package we also provide a small wrapper around the awesome [emcee](https://github.com/dfm/emcee) `EnsembleSampler` that allows us to store some inference-specific parameters before sampling and of course the samples themselves after sampling.

Let's start with the first part:

In [10]:
# plus one dimension for the late T-stage's time parameter
ndim = len(original_model.spread_probs) + 1

# number of concurrent walkers that sample the space
nwalkers = 10 * ndim

# define the log-likelihood
def log_prob_fn(theta, sys, early_p=0.3, max_t=10):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    t = np.arange(max_t + 1)
    time_dists={
        "early": lymph.utils.fast_binomial_pmf(t, max_t, early_p),
        "late" : lymph.utils.fast_binomial_pmf(t, max_t, late_p)
    }
    
    return sys.marginal_log_likelihood(
        spread_probs, t_stages=["early", "late"], time_dists=time_dists
    )

<div class="alert alert-warning">
    Warning 

    The provided log-likelihood function won't be stored anywhere! It is not possible to store arbitrary python code in an HDF5 file and retrieve it automatically in a safe manner.
</div>

## Sampling

For storing the results, we make use of the `HDFBackend` from `emcee`, while the sampling itself can be done any way one pleases. However, we have written a sampling method `run_sampling` that smartly samples until convergence.

<div class="alert alert-info">
    See also

    The creators of the `emcee` package have laid out how this "sampling to convergence" works in a [really nice tutorial](https://emcee.readthedocs.io/en/stable/tutorials/monitor/), which basically served as inspiration to the `run_sampling` method as well as our attempts of storing the model settings in an HDF5 file to begin with.
</div>

In [11]:
# this chain will surely be too short, but it doesn't matter here
max_steps = 200

# prepare the backend
backend = emcee.backends.HDFBackend(
    filename=demo_hdf_file,
    name="original/samples"
)
backend.reset(nwalkers, ndim)

# use Pool() from multiprocessing for parallelisation
with Pool() as pool:
    original_sampler = lymph.utils.EnsembleSampler(
        nwalkers, ndim, 
        log_prob_fn, args=[original_model], 
        pool=pool, backend=backend
    )
    acor_list = original_sampler.run_sampling(max_steps)

Starting sampling


100%|██████████| 200/200 [04:58<00:00,  1.49s/it]

Max. number of steps reached
Acceptance fraction = 19.89%
Mean autocorrelation time = 22.04


We can make sure the chain of samples is actually stored by trying to retrieve it from the HDF5 file directly:

In [12]:
test_backend = emcee.backends.HDFBackend(
    filename=demo_hdf_file,
    name="original/samples",
    read_only=True
)
test_backend.shape

(120, 12)

The first round has finished now. Let's see if we can repoduce all of that as intended.

## Do it all again

When we load a model instance from the HDF5 storage, all the settings, i.e. the graph, the diagnostic modalities and the loaded data, should still be the same as in the beginning. So let's check that with some `assert`s.

In [13]:
recovered_model = lymph.utils.system_from_hdf(
    filename=demo_hdf_file,
    name="original/model"
)

In [14]:
assert recovered_model.graph == graph, "Wrong graph!"
recovered_model.graph

{('tumor', 'primary'): ['I', 'II', 'III', 'IV'],
 ('lnl', 'I'): ['II'],
 ('lnl', 'II'): ['III'],
 ('lnl', 'III'): ['IV'],
 ('lnl', 'IV'): []}

In [15]:
assert recovered_model.modalities == diagnostic_spsn, "Wrong diagnostic modalities!"
recovered_model.modalities

{'MRI': [0.63, 0.81], 'PET': [0.86, 0.79]}

In [16]:
assert np.all(recovered_model.patient_data == synthetic_data), "Wrong data!"
recovered_model.patient_data

MRI                                                     PET         \
    contra                        ipsi                      contra          
         I     II    III     IV      I     II    III     IV      I     II   
0    False   True  False  False   True   True  False   True   True  False   
1    False  False  False  False  False   True   True   True  False  False   
2    False   True  False  False  False  False   True   True  False   True   
3     True  False  False  False  False   True   True  False  False  False   
4    False  False  False  False   True  False   True   True  False  False   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
195  False  False   True   True   True  False   True   True  False  False   
196   True  False   True  False  False   True   True  False  False   True   
197  False   True  False  False   True   True   True   True  False   True   
198   True  False   True   True   True   True  False  False  False  False   
199  False   True  False   True  False  False   True  False   True   True   

                                                 info  
                    ipsi                        tumor  
       III     IV      I     II    III     IV t_stage  
0    False  False  False   True   True   True    late  
1    False  False  False  False   True  False   early  
2    False  False  False  False  False   True    late  
3    False  False  False  False   True  False    late  
4    False  False  False  False   True   True    late  
..     ...    ...    ...    ...    ...    ...     ...  
195  False  False  False   True   True  False   early  
196   True  False  False   True  False  False    late  
197  False  False   True  False  False   True    late  
198  False  False  False  False  False  False   early  
199   True   True  False  False  False  False    late  

[200 rows x 17 columns]

The recovery worked! Since we want to do another sampling round which itself should be reproducable as well, we can immediately store the recovered model in a new group of the HDF5 file.

In [17]:
recovered_model.to_hdf(
    filename=demo_hdf_file,
    name="recovered/model"
)

/home/rmnldwg/repos/lymph/.venv/lib/python3.8/site-packages/tables/attributeset.py:291: DataTypeWarning: Unsupported type for attribute 'base_symmetric' in node 'model'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/home/rmnldwg/repos/lymph/.venv/lib/python3.8/site-packages/tables/attributeset.py:291: DataTypeWarning: Unsupported type for attribute 'trans_symmetric' in node 'model'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


Now for the `EnsembleSampler`. We can recover the number of walkers and dimension from the previously stored HDF5 file. Note that I use two backends: One for retrieving the shape of the stored chain, which accesses the HDF5 group of the original sampling round. The however, I set up a new group for the second sampling round. The reason for this is that I don't want to call the `reset` method of the backend on my stored samples, thereby deleting them.

In [18]:
tmp_backend = emcee.backends.HDFBackend(
    filename=demo_hdf_file,
    name="original/samples"
)
nwalkers, ndim = tmp_backend.shape

recovered_backend = emcee.backends.HDFBackend(
    filename=demo_hdf_file,
    name="recovered/samples"
)
recovered_backend.reset(nwalkers, ndim)

with Pool() as pool:
    recovered_sampler = lymph.utils.EnsembleSampler(
        nwalkers, ndim,
        log_prob_fn, args=[recovered_model],
        pool=pool, backend=recovered_backend
    )
    acor_list = recovered_sampler.run_sampling(max_steps)

Starting sampling


100%|██████████| 200/200 [09:03<00:00,  2.72s/it]

Max. number of steps reached
Acceptance fraction = 23.22%
Mean autocorrelation time = 22.14


As you can see, it required relatively few stops to reproduce a sampling round. Just don't forget the log-probability function, as that is hard to store anywhere in an HDF5 file.

Beyond that one can of course use the [h5py](https://docs.h5py.org/en/stable/) package or `pandas`' implemented capabilities to interact with the HDF5 file format to store and retrieve even more information, like a description of what was done or what exactly the log-likelihood does.